# Data Cleaning in SQL

# Access Database

In [1]:
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

import pickle
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.precision', 3)

In [2]:
plt.style.use('ggplot')
plt.rc('font', size=18)

In [3]:
# Postgres info to connect

connection_args = {
                   'host': 'localhost',  # Connecting to local version of psql
                   'dbname': 'fox_ticket_sales',    # DB that we are connecting to
                   'port': 5432          # port opened on AWS
                  }

connection = pg.connect(**connection_args)

In [4]:
cursor = connection.cursor()

# Data Cleaning

- There are a number of shows listed that have no ticket sales data. These will be removed from the dataset.
- There are a number of columns that are unnecessary for analysis and will be removed. See below:
    - gross_gate
    - currency
    - venue_name
    - venue_city
    - venue_state
    - venue_country
    - promoter
- **Assumptions**
    - For any concert that has a max ticket price of $0.00, assume that the price for all tickets is the minimum ticket price (i.e. min ticket price = max ticket price)
    - For co-headlining tours, there are multiple acts listed as headliners; the first act listed in the headliner column will be treated as the show's headliner.  For each co-headliner, add 2 to Num_Support column (i.e. co-headliners hold twice the weight of a regular supporting act). This will make it easier to interpret artist features later on.
    - For any show with more than 4 supporting acts, it is very likely to be billed as a "festival". A new feature called "show_type" will be created to label each show as a single headliner, co-headliner, or festival
- **Add the following time-based features**
    - Year
    - Month
    - Season
    - Day of Week
    - Time of Week (Weekday vs Weekend)

In [116]:
# drop_views = """    
#               DROP VIEW past_shows;
#               DROP TABLE IF EXISTs past_shows;
#               """

# cursor.execute(drop_views)
# cursor.execute('commit;')

In [117]:
past_shows_view = """ 
                  CREATE VIEW v_past_shows AS
                         WITH past_shows_cte AS (
                                                 SELECT start_date::date AS start_date,
                                                        end_date::date AS end_date,
                                                        DATE_PART('year', start_date::date)::int AS year,
                                                        DATE_PART('month', start_date::date)::int AS month,
                                                        DATE_PART('dow', start_date::date) AS day_of_week,
                                                        num_shows,
                                                        headliner,
                                                        support,
                                                        CASE WHEN support IS NOT NULL AND (LENGTH(headliner) - LENGTH(REPLACE(headliner, '/', '')) + 1 > 1)
                                                             THEN LENGTH(support) - LENGTH(REPLACE(support, ',', '')) + 1 + 2*(LENGTH(headliner) - LENGTH(REPLACE(headliner, '/', '')))
                                                             WHEN support IS NULL AND (LENGTH(headliner) - LENGTH(REPLACE(headliner, '/', '')) + 1 > 1)
                                                             THEN 2*(LENGTH(headliner) - LENGTH(REPLACE(headliner, '/', '')))
                                                             WHEN support IS NULL
                                                             THEN 0
                                                             ELSE LENGTH(support) - LENGTH(REPLACE(support, ',', '')) + 1
                                                         END AS num_support,
                                                        tickets_sold,
                                                        gross_usd,
                                                        venue_capacity,
                                                        percentage_sold,
                                                        ticket_price_min,
                                                        ticket_price_max
                                                   FROM ticket_sales
                                                  WHERE tickets_sold IS NOT NULL
                                                )

                  SELECT start_date,
                         end_date,
                         year,
                         TO_CHAR(start_date::date, 'Month') AS month,
                         CASE WHEN month IN (12,1,2) THEN 'Winter'
                              WHEN month IN  (3,4,5) THEN 'Spring'
                              WHEN month IN  (6,7,8) THEN 'Summer'
                              ELSE 'Autumn'
                               END AS season,
                         TO_CHAR(start_date::date, 'Day') AS day_of_week,
                         CASE WHEN day_of_week IN (0,1,2,3) THEN 'Weekday'
                              ELSE 'Weekend'
                               END AS time_of_week,                                
                         num_shows,
                         CASE WHEN num_support > 4
                              THEN 'festival'
                              WHEN num_shows > 1
                              THEN 'multiple shows'
                              WHEN LENGTH(headliner) - LENGTH(REPLACE(headliner, '/', '')) + 1 > 1
                              THEN 'co-headliner'
                              ELSE 'single headliner'
                               END AS show_type, 
                         REPLACE(SPLIT_PART(headliner, '/', 1), '"', '') AS headliner,
                         support,
                         num_support,
                         tickets_sold,
                         gross_usd,
                         venue_capacity,
                         percentage_sold,
                         ticket_price_min,
                         CASE WHEN ticket_price_max = 0
                              THEN ticket_price_min
                              ELSE ticket_price_max
                          END AS ticket_price_max
                    FROM past_shows_cte;
                  """

cursor.execute(past_shows_view)
cursor.execute('commit;')

In [118]:
query = """
        SELECT * 
          FROM v_past_shows;
        """

past_shows_df = pd_sql.read_sql(query, connection)

In [119]:
past_shows_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 751 entries, 0 to 750
Data columns (total 18 columns):
start_date          751 non-null object
end_date            751 non-null object
year                751 non-null int64
month               751 non-null object
season              751 non-null object
day_of_week         751 non-null object
time_of_week        751 non-null object
num_shows           751 non-null int64
show_type           751 non-null object
headliner           751 non-null object
support             579 non-null object
num_support         751 non-null int64
tickets_sold        751 non-null int64
gross_usd           751 non-null float64
venue_capacity      751 non-null int64
percentage_sold     751 non-null float64
ticket_price_min    751 non-null float64
ticket_price_max    751 non-null float64
dtypes: float64(4), int64(5), object(9)
memory usage: 105.7+ KB


# Save Final Table as Dataframe

- For use in "" notebook

In [120]:
with open('Data/sql_past_shows_df.pkl', 'wb') as to_write:
    pickle.dump(past_shows_df, to_write)